<a href="https://colab.research.google.com/github/Robert0912/Solucion_Sudokus_con_ANN/blob/main/Sudokus_Version2_ImpModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/drive/MyDrive/Documents/Universidad/sudoku2.csv')

feat_raw = data['quizzes']
label_raw = data['solutions']

feat = []
label = []

for i in feat_raw:

    x = np.array([int(j) for j in i]).reshape((9,9,1))
    feat.append(x)

feat = np.array(feat)
feat = feat/9
feat -= .5    

for i in label_raw:

    x = np.array([int(j) for j in i]).reshape((81,1)) - 1
    label.append(x)   

label = np.array(label)

del(feat_raw)
del(label_raw)    

X_train, X_test, Y_train, Y_test = train_test_split(feat, label, test_size=0.2)

In [ ]:
print(X_test.shape)
print(Y_test.shape)

print(X_train.shape)
print(Y_train.shape)

(200000, 9, 9, 1)
(200000, 81, 1)
(800000, 9, 9, 1)
(800000, 81, 1)


In [ ]:
import keras
from keras.layers import Activation
from keras.layers import BatchNormalization, Dense, Flatten, Reshape

model = keras.models.load_model("/content/drive/MyDrive/Documents/Universidad/modelo2.h5") 

In [ ]:
def norm(a):
    
  return (a/9)-.5

def denorm(a):
    
  return (a+.5)*9

import copy

def inference_sudoku(sample):
    
    feat = copy.copy(sample)
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

def accuracy(predicted_labels, actual_labels):
  
  predicted_labels = np.reshape(predicted_labels,(1,81))
  actual_labels = np.reshape(actual_labels,(1,81))
  diff = predicted_labels[0] - actual_labels[0]

  return (float(np.count_nonzero(diff==0)) / len(diff))   

def rendimiento(x,y,y_predict):

  diff = y_predict [[0]] - y[[0]]

  iguales_y_vs_ypredict=(float(np.count_nonzero(diff==0)))

  correctos_previos=np.count_nonzero(x[[0]])

  diferencia=iguales_y_vs_ypredict-correctos_previos

  if diferencia>0:

    return diferencia/((81)-correctos_previos)

  else:

    return diferencia/(correctos_previos)

In [ ]:
a = 100

acc = np.zeros(a)
pred = np.zeros((9,9,a))

for j in range(a):
  pred[:,:,j] = inference_sudoku(X_test[j,:,:,0])

print(pred.shape)

(9, 9, 100)


In [ ]:
acc = np.zeros(a)
ren = np.zeros(a)

for i in range(a):
  acc[i] = accuracy(pred[:,:,i], Y_test[i,:,0]+1)
  ren[i] = rendimiento(np.reshape(denorm(X_test[i,:,:,0]),(1,81)),np.reshape(Y_test[i,:,0]+1,(1,81)),np.reshape(pred[:,:,i],(1,81)))

print(sum(acc)/a)
print(max(acc))
print(min(acc))


print(sum(ren)/a)
print(max(ren))
print(min(ren))

0.9937037037037036
1.0
0.7530864197530864
0.9891330569256529
1.0
0.5918367346938775


In [ ]:
print(denorm(X_test[0,:,:,0]))
print(inference_sudoku(X_test[0,:,:,0]))
print(Y_test[0,:,0]+1)


[[8. 1. 2. 0. 9. 6. 4. 3. 0.]
 [6. 0. 5. 0. 3. 0. 0. 0. 8.]
 [4. 0. 3. 8. 2. 5. 1. 0. 0.]
 [0. 6. 0. 9. 5. 3. 0. 4. 0.]
 [9. 2. 0. 6. 0. 1. 7. 5. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 6.]
 [2. 0. 0. 0. 6. 7. 0. 0. 0.]
 [1. 0. 6. 4. 0. 0. 0. 0. 0.]
 [0. 0. 7. 3. 1. 2. 0. 9. 4.]]
[[8 1 2 7 9 6 4 3 5]
 [6 9 5 1 3 4 2 7 8]
 [4 7 3 8 2 5 1 6 9]
 [7 6 1 9 5 3 8 4 2]
 [9 2 8 6 4 1 7 5 3]
 [3 5 4 2 7 8 9 1 6]
 [2 4 9 5 6 7 3 8 1]
 [1 3 6 4 8 9 5 2 7]
 [5 8 7 3 1 2 6 9 4]]
[8 1 2 7 9 6 4 3 5 6 9 5 1 3 4 2 7 8 4 7 3 8 2 5 1 6 9 7 6 1 9 5 3 8 4 2 9
 2 8 6 4 1 7 5 3 3 5 4 2 7 8 9 1 6 2 4 9 5 6 7 3 8 1 1 3 6 4 8 9 5 2 7 5 8
 7 3 1 2 6 9 4]
